# Foundry - All Boiler Inspections

## Environment Setup

In [1]:
import difflib
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sys
sys.path.insert(0, '..')
import warnings
import zlib

# local
from libs.etl.data_cleaning import generate_aep_outage_dfs, generate_duke_outage_dfs, generate_iir_boilers_dfs, determine_component
from libs.inspection_analysis.utils import analyze_inspection_df, hist_inspection_df, check_thickness, get_thickness_histogram, group_critdat, get_inspection_df
from libs.interfaces.rolodex_interface import rolodex_load
from libs.reference.constants import COMPONENT_TYPE_MAP, POWER_CUST_LIST
from libs.viz.utils import add_vbar_labels, add_hbar_labels

## Settings

In [2]:
plt.style.use('dark_background')
#sns.set_style('darkgrid')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

warnings.filterwarnings('ignore')

wd = os.getcwd()
print(wd)

C:\Users\tom.fiorelli\Documents\GitHub\davos-data-project\notebooks


## Read in US Boiler data from IIR

In [3]:
us_boilers_df, operational_us_boilers_df = generate_iir_boilers_dfs()

In [4]:
us_boilers_df.shape

(2538, 53)

In [5]:
us_boilers_df.head(1)

,PARENTNAME,OWNER_NAME,OPER_NAME,PLANT_NAME,PHYS_CITY,P_COUNTY,UNIT_STATE,P_ST_NAME,P_COUNTRY,MARKET_REG,WORLD_REG,IND_CODE_DESC,SIC_CODE,PLANT_ID,UNIT_NAME,UNIT_ID,UNIT_TYPE,OUT_DESIGN,KO_DATE,STARTUP,STARTYEAR,SHUTDOWN,PRIM_FUEL,SECND_FUEL,FUEL_GROUP,HEAT_RATE,PWR_USAGE,U_STATUS,BOIL_NAME,BOILERTYPE,BMFR_NAME,BOIL_INFO,STEAM,STEAM_UOM,STEAM_TON,PRESSURE,PRES_UOM,PRES_BAR,TEMP,TEMP_UOM,TEMP_CEL,DESIGNFIRM,CONSTFIRM,MMBTU_HR,KO_YEAR,LATITUDE,LONGITUDE,COGEN_CHP,SHUTDOWN_dt,SHUTDOWN_yr,BOIL_CONFIG,BOIL_TYPE_Tag,Criticality
0,The Blackstone Group Incorporated,Lightstone Generation LLC,Lightstone Generation LLC,James Gavin Power Station,Cheshire,Gallia,OH,Ohio,U.S.A.,Great Lakes,North America,Power,4911,1010019,James Gavin 02,1003920,Steam Turbine,1300.0,NaN,1-Jul-75,1975,1-Dec-35,Coal Bituminous,Fuel Oil No. 2 Distillate Diesel,Coal,9799.0,Base Load,Operational,Boiler 02,Dry Bottom Boiler,Babcock & Wilcox,Opposed-Fired SCR LNB ESP-CS FGD-WS Trona,9775000.0,lbs/hr,4887.5,3685.0,PSIG,254.0,1010.0,deg F,543.0,AEP Pro Serv,AEP Pro Serv,NaN,NaN,38.9347,-82.1158,0,2035-12-01,2035.0,Opposed,Unknown,none


In [6]:
operational_us_boilers_df.to_csv('../data/outputs/operational_us_boilers.csv')

## Read in Rolodex inspection data 

In [7]:
data = rolodex_load(
    "inspection", 
    links=[
        "component", 
        "unit", 
        "site", 
        "organization"
    ], 
    filters=[
        {
            "type": "component", 
            "key": "portal.component_type", 
            "value": "1"
        }
    ], 
    ids=[], 
    tags={
    }, 
    limit=100000
)

In [8]:
boiler_inspections_df = pd.json_normalize(data)

In [9]:
boiler_inspections_df.shape

(840, 155)

In [10]:
boiler_inspections_df.head()

,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.tags.site,links.site.tags.organization,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,links.organization.tags.organization,links.unit.data.display_name,links.unit.tags.radar_demo_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.radar_demo_id,links.unit.data.values.color,links.unit.data.values.asset_type,links.unit.data.geometry.type,links.unit.data.geometry.coordinates,links.unit.data.components,links.unit.data.description,links.unit.data.manufactured_date,links.site.data.values.lost_mwh,links.site.data.values.plant_type,links.site.data.description,links.component.data

In [11]:
completed_boiler_inspections_df = boiler_inspections_df[boiler_inspections_df['entry.data.portal.status']=='completed']

In [12]:
sliced_inspection_df = completed_boiler_inspections_df.loc[:, ('entry.data.portal.slug', 'links.component.data.portal.wall_nominal_thickness')].reset_index()

sliced_inspection_df.rename(
    columns={
        'entry.data.portal.slug': 'slug',	
        'links.component.data.portal.wall_nominal_thickness': 'nominal',
    }, 
    inplace=True
)

sliced_inspection_df.drop(columns={'index'}, inplace=True)



In [13]:
clean_nominals_df = sliced_inspection_df[
    ~sliced_inspection_df['nominal'].isna() &
    (sliced_inspection_df['nominal']!=0) &
    (sliced_inspection_df['nominal'] < 1) &
    (sliced_inspection_df['nominal'] > .1)
]

In [14]:
clean_nominals_df.shape

(657, 2)

In [15]:
clean_nominals_df.reset_index(inplace=True)

In [16]:
histogram_df, hist_error_slug_list = get_thickness_histogram(clean_nominals_df)

0/657
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
10/657
Error: No plots record on req.json ...
20/657
30/657
40/657
Error: No plots record on req.json ...
50/657
60/657
70/657
80/657
90/657
100/657
110/657
Error: No plots record on req.json ...
Error: No plots record on req.json ...
120/657
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
130/657
Error: No plots record on req.json ...
140/657
Error: No plots record on req.json ...
150/657
Error: No plots record on req.json ...
160/657
Error: No plots record on req.json ...
170/657
Error: No plots record on req.json ...
Error: No plots record on req.json ...
180/657
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
190/657
Error: No plots record on req.js

In [17]:
histogram_df.shape

(539, 25)

In [18]:
clean_nominals_df.shape

(657, 3)

In [19]:
len(hist_error_slug_list)

118

In [20]:
hist_error_slug_list

['20160414-c08dfd',
 '20160914-808361',
 '20170425-33a954',
 '20170427-4b31bf',
 '20180107-000000',
 '20181115-fd457e',
 '20181121-e59de2',
 '20190220-8bcb14',
 '20190222-3a5650',
 '20190225-3a0b57',
 '20190225-d91bce',
 '20190221-3cb34c',
 '20190322-b40ed8',
 '20190412-d45acf',
 '20220922-123456',
 '20190528-5b96bb',
 '20190528-c6c450',
 '20190621-6811d7',
 '20190716-e19fed',
 '20190718-4d5176',
 '20190718-a9cd3c',
 '20190705-ab215d',
 '20190730-465e41',
 '20190730-6c4c88',
 '20190730-825623',
 '20190730-996903',
 '20190829-a2dbf8',
 '20200126-40eff6',
 '20200623-5e4a3d',
 '20200603-a6ce00',
 '20200623-ce0451',
 '20200624-2e1086',
 '20200624-40b82f',
 '20200803-e72eb5',
 '20200813-bae141',
 '20200903-1eb2f2',
 '20200903-42f230',
 '20200917-25bf86',
 '20200925-4ea5d3',
 '20201016-1a2be4',
 '20201217-01ddb3',
 '20210302-35d5f8',
 '20210316-cbbde1',
 '20210714-0031c9',
 '20210708-f3509e',
 '20210831-5abaaf',
 '20211007-0e5af9',
 '20211102-8282ba',
 '20211130-cf1f82',
 '20211210-898ae2',


In [21]:
histogram_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss
0,20161025-3219a1,0.21,0.11300,0.2600,374,27169,2,374,208,31,6,0,0,0,0,0,14484,1319,59,6,0,0,0,0,0
1,20170311-e152db,0.23,4.51000,59.8400,342,24787,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20170316-e4aa87,0.25,0.08240,0.2726,283,9745,877,283,283,271,209,128,37,0,0,0,9080,5528,2311,855,255,54,0,0,0
3,20170317-e033fe,0.25,0.08350,0.2680,199,6871,432,199,195,159,119,79,30,0,0,0,5223,2157,651,297,122,32,0,0,0
4,20170327-53a53b,0.17,0.16008,0.2970,480,19797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
comb_df = histogram_df.merge(completed_boiler_inspections_df, how='left', left_on='slug', right_on='entry.data.portal.slug')

In [23]:
comb_df.shape

(539, 180)

In [24]:
IIR_ID_all_df = pd.read_csv('../data/outputs/mapped_us_boilers_12523.csv')

In [25]:
IIR_unit_map_df = IIR_ID_all_df.loc[:,('IIR_UNIT_ID', 'Unit.slug')].drop_duplicates()
IIR_unit_map_df.shape

(163, 2)

In [26]:
iir_mapped_comb_df = comb_df.merge(IIR_unit_map_df, how='left', left_on='entry.tags.unit', right_on='Unit.slug')

In [27]:
enriched_boiler_inspections_df = iir_mapped_comb_df.merge(us_boilers_df, how='left', left_on='IIR_UNIT_ID', right_on='UNIT_ID')

In [28]:
enriched_boiler_inspections_df.shape

(625, 235)

In [29]:
enriched_boiler_inspections_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.tags.site,links.site.tags.organization,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,links.organization.tags.organization,links.unit.data.display_name,links.unit.tags.radar_demo_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_custom

In [30]:
enriched_boiler_inspections_df.groupby(['BOIL_CONFIG'])['UNIT_ID'].nunique()

BOIL_CONFIG
Arch           1
Cyclone        3
Front         21
Opposed       25
Stoker         1
Tangential    41
Unknown       15
Name: UNIT_ID, dtype: int64

In [31]:
enriched_boiler_inspections_df.groupby(['BOIL_TYPE_Tag'])['UNIT_ID'].nunique()

BOIL_TYPE_Tag
CFB         7
PC         63
Unknown    36
Name: UNIT_ID, dtype: int64

In [32]:
enriched_boiler_inspections_df['Component_type'] = enriched_boiler_inspections_df['entry.tags.component'].astype(str).apply(lambda x: determine_component(x))

In [33]:
enriched_boiler_inspections_df['Power_Cust'] = enriched_boiler_inspections_df['links.organization.data.portal.name'].isin(POWER_CUST_LIST)

In [34]:
#set(enriched_boiler_inspections_df['entry.tags.component'])

In [35]:
enriched_boiler_inspections_df.groupby(['Component_type'])['slug'].nunique()

Component_type
Arch                             20
Backpass                         35
Bullnose                         18
Division Wall                    28
Slope                            62
Wall                            257
baseline-tube                     1
boiler                            4
boiler-roof                       8
cfb-boiler                        2
demo-panel                        2
final-exam                        6
floor                             6
front                             1
front-panel-12-redo               1
furnace-upper-and-lower           6
giovanni-mattucci                 1
header                            1
heater-1                          1
heater-2                          1
horizontal-superheat-bundles      1
left-4                            1
left-5                            1
left-6                            1
left-7                            1
left-8                            1
left-9                            1
left-panel-29

In [36]:
enriched_boiler_inspections_df.groupby(['Power_Cust'])['slug'].nunique()

Power_Cust
False    325
True     214
Name: slug, dtype: int64

In [37]:
enriched_boiler_inspections_df.to_csv('../data/outputs/enriched_boiler_inspections_df.csv')

Level?:

SCR
SOFA
SNCR

true / false ?

LNB = Low NOX Burners

FGR = Flue Gas Recirculation

BFB = bubbling fluidized bed

BAG ? - baghouse for bagging emissions



In [38]:
rich_df.to_csv('rich.csv')

NameError: name 'rich_df' is not defined

In [ ]:
rich_df[rich_df['Component_type']=='Other'].shape

In [ ]:
#rich_df['crits_per10k']

In [ ]:
rich_df.head(1)

In [ ]:
rich_df.loc[:,('Customer.name',	'Location.name')].drop_duplicates().sort_values(['Customer.name',	'Location.name']).to_csv('../data/intermediate/distinct_plants.csv')

In [ ]:
IIR_ID_slopes_df = pd.read_csv('../data/intermediate/slopes_wIIR_id.csv')

In [ ]:
IIR_mapping = IIR_ID_slopes_df.loc[:, ('IIR_UNIT_ID', 'Unit.slug')]

In [ ]:
iir_map_comb_df = comb_df.merge(IIR_mapping, how='left', on='Unit.slug')

In [ ]:
iir_map_comb_df.shape

In [ ]:
fin_df = iir_map_comb_df.merge(us_boilers_df, how='left', left_on='IIR_UNIT_ID', right_on='UNIT_ID')

In [ ]:
fin_df.shape

In [ ]:
fin_df['tube_ft_inspected'] = fin_df['bins_collected']*(3/12)

In [ ]:
fin_df.info()

In [ ]:
fin_df.columns

In [ ]:
trimmed_fin_df = fin_df.loc[:, (
    'Customer.name',
    'Location.name',
    'Unit.slug',
    'Component.slug',
    'slug',
    'nominal',
    'Component_type', 
    'Power_Cust',
    'tubes_inspected', 
    'bins_collected',
    'tube_ft_inspected',
    'crits_per10k',
    'Tubes w 10% Loss',
    'Tubes w 20% Loss',
    'Tubes w 30% Loss',
    'Tubes w 40% Loss',
    'Tubes w 50% Loss',
    'Tubes w 60% Loss',
    'Tubes w 70% Loss',
    'Tubes w 80% Loss',
    'Tubes w 90% Loss',
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'IIR_UNIT_ID',
    'P_ST_NAME',
    'MARKET_REG',
    'IND_CODE_DESC',
    'SIC_CODE',
    'PLANT_ID',
    'UNIT_TYPE', 
    'OUT_DESIGN', 
    'KO_DATE', 
    'STARTUP', 
    'STARTYEAR',
    'SHUTDOWN',
    'PRIM_FUEL',
    'SECND_FUEL',
    'FUEL_GROUP',
    'HEAT_RATE',
    'PWR_USAGE',
    'U_STATUS',
    'BOIL_NAME', 
    'BOILERTYPE', 
    'BMFR_NAME',
    'BOIL_INFO', 
    'STEAM', 
    'STEAM_UOM', 
    'STEAM_TON', 
    'PRESSURE', 
    'PRES_UOM',
    'PRES_BAR', 
    'TEMP', 
    'TEMP_UOM', 
    'TEMP_CEL', 
    'DESIGNFIRM', 
    'CONSTFIRM',
    'MMBTU_HR', 
    'KO_YEAR', 
    'LATITUDE', 
    'LONGITUDE', 
    'COGEN_CHP',
    'SHUTDOWN_dt', 
    'SHUTDOWN_yr'
)]

In [ ]:
trimmed_fin_df['Percentile_Rank'] = trimmed_fin_df.crits_per10k.rank(pct=True)

In [ ]:
trimmed_fin_df.to_csv('../data/outputs/enriched_boilers_df.csv')

In [ ]:
trimmed_fin_df.shape

In [ ]:
trimmed_fin_df.head()

In [ ]:
trimmed_fin_df.crits_per10k.describe()

In [ ]:
trimmed_fin_df.crits_per10k.median()

In [ ]:
np.percentile(trimmed_fin_df.crits_per10k, 90)

In [ ]:
np.percentile(trimmed_fin_df.crits_per10k, 81)

In [ ]:
np.percentile(trimmed_fin_df.crits_per10k, 50)

In [ ]:
trimmed_fin_df[trimmed_fin_df['Customer.name']=='Duke Energy']

In [ ]:
filtered_df = trimmed_fin_df[
    (trimmed_fin_df['FUEL_GROUP']=='Coal') &
    (trimmed_fin_df['PRIM_FUEL']=='Coal Bituminous') &
    (trimmed_fin_df['Component_type']=='Slopes') &
    (trimmed_fin_df['nominal']<=0.24) &
    (trimmed_fin_df['Power_Cust']==True)
              ]

In [ ]:
filtered_df.shape

In [ ]:
filtered_df.groupby(['Component_type'])[
    'nominal',
    'tubes_inspected', 
    'bins_collected',
    'tube_ft_inspected',
    'crits_per10k',
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
].median()

In [ ]:
filtered_df.sort_values(by=['crits_per10k'], ascending=False)

In [ ]:
filtered_df['Percentile_Rank'] = filtered_df.crits_per10k.rank(pct=True)

In [ ]:
filtered_df

In [ ]:
filtered_df.to_csv('../data/outputs/ranked_us_boilers.csv')

In [ ]:
gibson1slopes_df = get_inspection_df('20221004-565f7b')

In [ ]:
nominal = 0.203
min_t, max_t, tubes_inspected, bins_collected, tube_hist_bin_counts, bin_hist_bin_counts = hist_inspection_df(gibson1slopes_df, nominal)

In [ ]:
tube_hist_bin_counts

In [ ]:
bin_hist_bin_counts

In [ ]:
sum(bin_hist_bin_counts)

In [ ]:
bins_collected

In [ ]:
round((sum(bin_hist_bin_counts)/bins_collected)*10000)

In [ ]:
np.percentile(filtered_df.crits_per10k, 91)